In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet101  # 仅修改此处
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
import seaborn as sns
from tqdm import tqdm
import random

# 设置随机种子保证可重复性
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 自定义数据集类（保持不变）
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append((os.path.join(cls_dir, img_name), self.class_to_idx[cls_name]))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# 数据增强变换（完全保留原始代码）
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 创建数据集和数据加载器（路径需自行确认）
train_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset2\train', transform=train_transform)
val_dataset = CustomDataset(r'C:\Users\Lenovo\Desktop\dataset2\val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

# 修改模型定义部分（仅替换ResNet50为预训练ResNet101）
class ResNet101(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet101, self).__init__()
        # 加载预训练的ResNet101并修改最后的全连接层
        self.model = resnet101(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)  # 修改输出层为2分类
    
    def forward(self, x):
        return self.model(x)

# 初始化模型、损失函数和优化器
model = ResNet101(num_classes=2).to(device)  # 仅修改此处
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.000015)

# 训练和验证函数（完全保留原始代码）
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(dataloader)
    train_acc = 100 * correct / total
    
    return train_loss, train_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    val_loss = running_loss / len(dataloader)
    val_acc = 100 * correct / total
    
    cm = confusion_matrix(all_labels, all_preds)
    
    if len(np.unique(all_labels)) == 2:
        roc_auc = roc_auc_score(all_labels, all_probs)
    else:
        roc_auc = float('nan')
    
    return val_loss, val_acc, cm, roc_auc

# 训练循环（完全保留原始代码）
num_epochs = 500
best_val_acc = 0.0
best_model_state = None
train_losses = []
train_accs = []
val_losses = []
val_accs = []
val_roc_aucs = []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    val_loss, val_acc, cm, roc_auc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    val_roc_aucs.append(roc_auc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()
        torch.save(best_model_state, 'best_model.pth')
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Val ROC AUC: {roc_auc:.4f}" if not np.isnan(roc_auc) else "Val ROC AUC: N/A")

# 绘制训练和验证的损失曲线
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

# 绘制训练和验证的准确率曲线
plt.subplot(1, 3, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Curve')
plt.legend()

# 绘制ROC AUC曲线
plt.subplot(1, 3, 3)
plt.plot(val_roc_aucs, label='Validation ROC AUC')
plt.xlabel('Epoch')
plt.ylabel('ROC AUC')
plt.title('ROC AUC Curve')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves.png')
plt.show()

# 加载最佳模型
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# 在验证集上计算指标
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs.data, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

# 计算混淆矩阵
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=train_dataset.classes, 
            yticklabels=train_dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

# 计算分类报告
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

# 计算ROC曲线和AUC
if len(np.unique(all_labels)) == 2:
    fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig('roc_curve.png')
    plt.show()

# 计算敏感度、特异性、F1分数
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes, output_dict=True)

# 提取每个类别的指标
class_metrics = {}
for i, class_name in enumerate(train_dataset.classes):
    class_metrics[class_name] = {
        'precision': report[class_name]['precision'],
        'recall': report[class_name]['recall'],
        'f1-score': report[class_name]['f1-score'],
        'support': report[class_name]['support']
    }

# 计算宏平均和加权平均
macro_avg = {
    'precision': report['macro avg']['precision'],
    'recall': report['macro avg']['recall'],
    'f1-score': report['macro avg']['f1-score']
}

weighted_avg = {
    'precision': report['weighted avg']['precision'],
    'recall': report['weighted avg']['recall'],
    'f1-score': report['weighted avg']['f1-score']
}

# 计算马修相关系数 (MCC)
def calculate_mcc(y_true, y_pred, num_classes):
    cm = confusion_matrix(y_true, y_pred)
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tp = cm[1, 1]
    
    numerator = tp * tn - fp * fn
    denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    if denominator == 0:
        return 0
    return numerator / denominator

mcc = calculate_mcc(all_labels, all_preds, len(train_dataset.classes))

# 输出结果
print(f"\nBest Validation Accuracy: {best_val_acc:.2f}%")
print(f"Macro Average F1-score: {macro_avg['f1-score']:.4f}")
print(f"Weighted Average F1-score: {weighted_avg['f1-score']:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

# 输出每个类别的敏感度(召回率)和特异性
for i, class_name in enumerate(train_dataset.classes):
    tn = cm[0, 0] if i == 0 else cm[i, :i].sum() + cm[i, i+1:].sum()
    fp = cm[i, 0] if i == 0 else cm[0:i, i].sum() + cm[i+1:, i].sum()
    fn = cm[0, i] if i == 0 else cm[i, 0:i].sum() + cm[i, i+1:].sum()
    tp = cm[i, i]
    
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    
    print(f"\nClass: {class_name}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-score: {class_metrics[class_name]['f1-score']:.4f}")

Using device: cuda


D:\Anaconda\anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.78it/s]


Train Loss: 0.6580, Train Acc: 59.05%
Val Loss: 0.6376, Val Acc: 62.07%
Val ROC AUC: 0.6600

Epoch 2/500


...

Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.08it/s]


Train Loss: 0.0756, Train Acc: 98.52%
Val Loss: 0.7497, Val Acc: 76.55%
Val ROC AUC: 0.8014

Epoch 19/500


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.77it/s]


Train Loss: 0.0493, Train Acc: 98.81%
Val Loss: 0.6550, Val Acc: 77.24%
Val ROC AUC: 0.8159

Epoch 20/500


Validating:  63%|████████████████████████████████████████████▏                         | 12/19 [00:02<00:01,  5.97it/s]